In [16]:
Pkg.add("LightGraphs")
Pkg.add("TikzGraphs")
Pkg.add("Combinatorics")

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [17]:
using Pkg
using LightGraphs
using TikzGraphs
using Combinatorics

In [220]:
# n is the value of the labels, i.e, the n of T_n
function get_splits(n)
    rng = 0:n
    rng_set = Set(rng)
    # We use sets because with this approach is almost sure that we are generating duplicated splits
    needed_subsets = map(dim -> map(Set, Combinatorics.combinations(rng, dim)), 2:ceil(Int, (n+1)/2))
    splits = Set(map(subset -> Set([subset, symdiff(rng_set, subset)]), Iterators.flatten(needed_subsets)))
    return splits
end

get_splits (generic function with 1 method)

In [268]:
splits = get_splits(4)

Set{Set{Set{Int64}}} with 10 elements:
  Set(Set{Int64}[Set([4, 1]), Set([0, 2, 3])])
  Set(Set{Int64}[Set([0, 4, 1]), Set([2, 3])])
  Set(Set{Int64}[Set([4, 3]), Set([0, 2, 1])])
  Set(Set{Int64}[Set([0, 1]), Set([4, 2, 3])])
  Set(Set{Int64}[Set([4, 3, 1]), Set([0, 2])])
  Set(Set{Int64}[Set([2, 3, 1]), Set([0, 4])])
  Set(Set{Int64}[Set([4, 2]), Set([0, 3, 1])])
  Set(Set{Int64}[Set([0, 3]), Set([4, 2, 1])])
  Set(Set{Int64}[Set([3, 1]), Set([0, 4, 2])])
  Set(Set{Int64}[Set([0, 4, 3]), Set([2, 1])])